In [64]:
# Load required libraries
import os
import json
from datetime import datetime

# Define memory log path
memory_log_path = "../memory/session_log.json"

# Ensure memory file exists
if not os.path.exists(memory_log_path):
    with open(memory_log_path, "w", encoding="utf-8") as f:
        json.dump([], f, indent=2)

# Define function to log conversational turns
def log_conversation_entry(user_input, ai_response, emotional_tags=None):
    entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "ai_response": ai_response,
        "emotional_tags": emotional_tags or []
    }

    # Load existing log
    with open(memory_log_path, "r", encoding="utf-8") as f:
        log = json.load(f)

    # Append new entry
    log.append(entry)

    # Save updated log
    with open(memory_log_path, "w", encoding="utf-8") as f:
        json.dump(log, f, indent=2)


In [65]:
# Ensure memory directory exists
os.makedirs("memory", exist_ok=True)


In [66]:
# Basic placeholder AI response function
def simulate_ai_response(user_input):
    """
    This is a stub to simulate AI response generation.
    Replace this logic with a real model or API call as needed.
    """
    if "help" in user_input.lower():
        return "I'm here to help. Tell me more about what's going on."
    elif "tired" in user_input.lower():
        return "It sounds like you've had a long day. Want to talk about it?"
    else:
        return "Thanks for sharing. What else is on your mind?"


In [67]:
# Predefined simple emotional tag dictionary
emotional_keywords = {
    "help": "supportive",
    "tired": "empathy",
    "thanks": "gratitude",
    "sorry": "remorse",
    "happy": "joy",
    "angry": "frustration",
    "worried": "concern",
    "love": "affection",
    "hate": "aversion"
}

# Emotional tagger based on keywords in the AI response
def tag_emotional_content(text):
    tags = []
    for word, tag in emotional_keywords.items():
        if word in text.lower():
            tags.append(tag)
    return tags


In [68]:
def process_conversation(user_input):
    ai_response = simulate_ai_response(user_input)
    tags = tag_emotional_content(ai_response)
    
    entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "ai_response": ai_response,
        "emotional_tags": tags
    }

    append_to_memory(entry)

    print(f"\n🤖 ANGEL.AI: {ai_response}")
    if tags:
        print(f"🧭 Emotional Tags: {', '.join(tags)}")
    else:
        print("🧭 Emotional Tags: None")

# Optional: trigger an example
# process_conversation("I'm feeling a little tired but grateful for your help.")


In [69]:
# Load emotional tags from memory
def load_emotional_tags(memory_path):
    if os.path.exists(memory_path):
        with open(memory_path, "r") as f:
            try:
                tags = json.load(f)
                return tags
            except json.JSONDecodeError:
                return []
    return []

# Analyze recent emotional tags to find dominant emotional tone
def get_dominant_emotion(tags, window_size=5):
    if not tags:
        return "neutral"
    
    recent_tags = tags[-window_size:]
    emotion_scores = {}

    for tag in recent_tags:
        emotion = tag[1] 
        emotion_scores[emotion] = emotion_scores.get(emotion, 0) + 1

    dominant_emotion = max(emotion_scores, key=emotion_scores.get)
    return dominant_emotion

# Apply this to current session
memory_path = os.path.join("..", "data", "emotional_tags.json")
emotional_tags = load_emotional_tags(memory_path)
dominant_emotion = get_dominant_emotion(emotional_tags)

print(f"🔄 Dominant emotional tone in recent exchanges: {dominant_emotion}")


🔄 Dominant emotional tone in recent exchanges: neutral


In [70]:
# Dictionary of emotional keywords and their weights
emotional_tags = {
    "overwhelmed": 0.9,
    "frustrated": 0.8,
    "angry": 0.85,
    "anxious": 0.75,
    "sad": 0.7,
    "worried": 0.65,
    "believe": 0.4,
    "hope": 0.3,
    "calm": 0.2,
    "okay": 0.1,
    "neutral": 0.0,
    "confident": -0.3,
    "excited": -0.5,
    "relieved": -0.4,
    "grateful": -0.6,
    "happy": -0.7,
    "joyful": -0.9,
    "inspired": -1.0
}


In [71]:
def tag_emotional_tokens(text):
    tokens = text.split()
    weighted_tokens = []
    for token in tokens:
        weight = emotional_tags.get(token.lower(), 0.0)
        weighted_tokens.append((token, weight))
    return weighted_tokens


In [72]:
def save_to_session_log(log_entry, file_path="memory/session_log.json"):
    try:
        # Load existing log or start new one
        if os.path.exists(file_path):
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        else:
            data = []

        data.append(log_entry)

        # Save updated log
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4)

        print("✅ Session log updated.")
    except Exception as e:
        print(f"⚠️ Error saving to session log: {e}")


In [73]:
def summarize_emotions(weighted_tokens):
    """
    Summarizes the emotional weights and returns a dominant tone.
    """
    emotion_totals = {}
    for emotion, weight in weighted_tokens:
        if emotion not in emotion_totals:
            emotion_totals[emotion] = 0
        emotion_totals[emotion] += weight

    # Sort by total weight descending
    sorted_emotions = sorted(emotion_totals.items(), key=lambda x: x[1], reverse=True)

    if sorted_emotions:
        dominant_emotion = sorted_emotions[0][0]
        print(f"🔍 Dominant emotional tone in recent exchanges: {dominant_emotion}")
        return dominant_emotion
    else:
        print("🔍 No dominant emotion detected.")
        return "neutral"


In [74]:
# Adaptive AI response generator based on dominant emotional tone

def generate_adaptive_response(user_input, dominant_emotion):
    """
    Simulates an AI response that adapts to the emotional context.
    """
    if dominant_emotion == "stress":
        response = f"I can sense some tension in our recent exchange. Let's take things one step at a time. You're doing great. How can I support you right now?"
    elif dominant_emotion == "confusion":
        response = f"It seems like something might not be clear. Would it help if I explained that another way or broke it down further?"
    elif dominant_emotion == "joy":
        response = f"I'm glad you're feeling positive! Let's keep this momentum going — what would you like to explore next?"
    elif dominant_emotion == "curiosity":
        response = f"You're asking excellent questions. Let's dig deeper into that together — here's what I think:"
    elif dominant_emotion == "neutral":
        response = f"Okay — here's my best response based on what you've asked:"
    else:
        response = f"I'm here with you, whatever you're feeling. Let's keep going — how can I assist?"

    return response

# Example simulation:
user_input = "I’m trying to understand how the feedback loop affects response."
adaptive_response = generate_adaptive_response(user_input, dominant_emotion)
print("\n🤖 Adaptive AI response:")
print(adaptive_response)



🤖 Adaptive AI response:
Okay — here's my best response based on what you've asked:


In [76]:
# Generate AI response based on emotional tone
def generate_ai_response(user_input, emotion):
    if emotion == "positive":
        return "I'm glad to hear that! Let's keep up the good work together."
    elif emotion == "negative":
        return "That sounds tough. I'm here to help however I can."
    elif emotion == "neutral":
        return "Got it. Let’s keep things moving forward."
    elif emotion == "anxious":
        return "I can feel the pressure you're under. Let’s break things down together."
    elif emotion == "hopeful":
        return "Hope is powerful — let’s build on that together."
    else:
        return "Thanks for sharing. Let’s keep the momentum going."


In [77]:
run_conversation_step("I feel totally overwhelmed trying to build this project, but I believe in it.")


✅ Session log updated.
🔎 Detected emotional tone: 0.0
✅ Session log updated.
🤖 Adaptive AI response:
Thanks for sharing. Let’s keep the momentum going.


'Thanks for sharing. Let’s keep the momentum going.'